In [99]:
import tkinter as tk
from tkinter import filedialog as fd
import cv2 as cv
import numpy as np
from PIL import Image, ImageTk
import os
import openpyxl

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [100]:
class Canvas():
    
    def __init__(self,window):
        self.window = window
        self.canvas1 = tk.Canvas(self.window, width=1200, height=800, bg='white')
        self.canvas2 = tk.Canvas(self.window, width=340, height=495, bg='white')
        self.canvas3 = tk.Canvas(self.window, width=340, height=300, bg='white')
        self.canvas1.create_rectangle(2, 3, 1195, 795, outline='black', width=2)
        self.canvas2.create_rectangle(5,3,330,490,outline='black',width=2)
        self.canvas3.create_rectangle(5,3,330,295,outline='black',width=2)
        self.canvas2.create_text(150,15,text="List",font=("Arial", 12))
        self.canvas1.create_text(600,25,text="NHẬN DIỆN KHUÔN MẶT",font=("Arial", 25))
        self.canvas1.place(x=0,y=0)
        self.canvas2.place(x=1200,y=0)
        self.canvas3.place(x=1200,y=500)
        self.canvas1.create_line(5, 50, 1194, 50, fill="black", width=2)
        self.canvas3.create_line(7, 150, 328, 150, fill="black", width=2)
        self.canvas3.create_text(160,135,text="Add Person",font=("Arial", 12))
    def __init__1(self):
        self.canvas1 = tk.Canvas(self.window, width=1200, height=800, bg='white')
        self.canvas1.create_rectangle(2, 3, 1195, 795, outline='black', width=2)
        self.canvas1.create_text(600,25,text="NHẬN DIỆN KHUÔN MẶT",font=("Arial", 25))
        self.canvas1.place(x=0,y=0)
        self.canvas1.create_line(5, 50, 1194, 50, fill="black", width=2)

        self.canvas2 = tk.Canvas(self.window, width=340, height=495, bg='white')
        self.canvas2.create_rectangle(5,3,330,490,outline='black',width=2)
        self.canvas2.place(x=1200,y=0)
        self.canvas2.create_text(150,15,text="List",font=("Arial", 12))
        # self.canvas1.create_line(1202, 500, 1528, 500, fill="black", width=2)
    def clear_console(self):
        self.canvas1.delete('all')
        self.canvas2.delete('all')
        self.__init__1()
        
    def getCanvas1(self):
        return self.canvas1
    def getCanvas2(self):
        return self.canvas2

In [101]:
class interface():
    def __init__(self):
        self.window = tk.Tk()
        # setting attribute
        self.window.attributes('-fullscreen', True)
        self.window.title("Nhận diện người")

        
        self.canvas= Canvas(self.window)

        self.entry = tk.Entry(self.window)
        self.entry.place(x=1310,y=512)

        # self.button0 = tk.Button(self.window,text= "From picture",command= lambda: add_person().pick_picture())
        # self.button0.place(x=1330,y=580)
        
        self.button1 = tk.Button(self.window, text="Pick picture",command=lambda:show_image(self.window,self.canvas.getCanvas2()).show_image())
        self.button1.place(x=600,y=810)

        self.button2 = tk.Button(self.window,text="   Capture   ",command=lambda:button2_clicked(self.window,self.entry,self.canvas))
        self.button2.place(x=1220,y=510)

        self.button3 = tk.Button(self.window,text="    Clear    ",command=lambda: self.canvas.clear_console())
        self.button3.place(x=1350 ,y=700)

        self.button4 = tk.Button(self.window,text="    Close    ",command= lambda:self.close())
        self.button4.place(x=1350,y=765)

        # Chạy vòng lặp chính
        self.window.mainloop()
    def close(self):
        self.window.destroy()

In [102]:
def write_file(list_MSSV,file_path):

    # Đọc dữ liệu từ file Excel
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    data = []
    print(list_MSSV)
    for row in sheet.iter_rows(min_row=3, values_only=True):
        # print(row,end="\n")
        stt, MSSV, Ho, Ten, Lop, diem_danh,*empty_value = row
        # print(Ten)
        if MSSV in list_MSSV:
            diem_danh+=1
        data.append({"stt": stt, "Mã sinh viên": MSSV, "Họ": Ho, "Tên": Ten, "Lớp":Lop, "Điểm danh": diem_danh})
    # Tạo một workbook mới
    workbook = openpyxl.Workbook()
    sheet = workbook.active

    # Ghi tiêu đề cột
    sheet.append(["TT", "MSSV", "Họ", "tên sinh viên","Lớp","Điểm danh" ])

    # Ghi dữ liệu vào sheet
    for entry in data:
        sheet.append([entry["stt"], entry["Mã sinh viên"], entry["Họ"], entry["Tên"], entry["Lớp"],entry["Điểm danh"]])

    # Lưu workbook thành file Excel mới
    workbook.save(file_path)


In [103]:
class train():
    def __init__(self):
        self.base_folder = "./Dataset1/"
        # Khởi tạo danh sách để lưu trữ hình ảnh và nhãn (labels)
        self.images = []
        self.labels = []
        # Khởi tạo một từ điển để ánh xạ từ tên nhãn (string) sang số nguyên
        self.label_to_int = {}
        self.int_to_label = {}
        self.current_label = 0
        self.create_data()
        self.model()
    def create_data(self):
        # Duyệt qua tất cả các thư mục con trong thư mục chính
        for person_folder in os.listdir(self.base_folder):
            person_path = os.path.join(self.base_folder, person_folder) 
            # Kiểm tra nếu person_path là một thư mục
            if os.path.isdir(person_path):
                # Lặp qua tất cả các tệp trong thư mục của người đó
                for filename in os.listdir(person_path):
                    # Kiểm tra nếu tệp có định dạng hình ảnh (vd: jpg, jpeg, png)
                    if filename.endswith((".jpg", ".jpeg", ".png")):
                        # Tạo đường dẫn đầy đủ đến tệp
                        image_path = os.path.join(person_path, filename)
                        # Đọc hình ảnh bằng OpenCV
                        image = cv.imread(image_path)
                        if image is not None:
                            # Chuyển hình ảnh thành ma trận và thêm vào danh sách images
                            # Thay đổi kích thước hình ảnh (vd: 50*50)
                            image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
                            image = cv.resize(image, (128, 128))
                            self.images.append(image)
                            
                            # Kiểm tra xem nhãn này đã được thêm vào từ điển chưa
                            if person_folder not in self.label_to_int:
                                self.label_to_int[person_folder] = self.current_label
                                self.int_to_label[self.current_label] = person_folder
                                self.current_label += 1
                            # Chuyển đổi tên nhãn từ chuỗi sang số nguyên và thêm vào danh sách labels
                            label = self.label_to_int[person_folder]
                            self.labels.append(label)

        # Chuyển danh sách images và labels thành các mảng NumPy
        images_array = np.array(self.images)
        labels_array = np.array(self.labels)

        # Chuẩn hóa giá trị pixel để nằm trong khoảng [0, 1]
        images_array = images_array / 255.0
        self.train_images = images_array
        self.train_labels = labels_array
    def model(self):
        model = models.Sequential()
        model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.Flatten())
        model.add(layers.Dense(64, activation='relu'))

        # Thêm Dropout để giảm overfitting
        model.add(layers.Dropout(0.5))
        model.add(layers.Dense(self.current_label, activation='softmax'))  # Số lượng lớp đầu ra bằng số nhãn, sử dụng softmax
        model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Sử dụng sparse categorical cross-entropy cho việc phân loại
              metrics=['accuracy'])

        history = model.fit(self.train_images, self.train_labels, epochs=10)  # Tăng số epoch để huấn luyện lâu hơn
                            # validation_data=(self.test_images, self.test_labels))
        model.save("my_model.keras")

In [104]:
class add_person():
    def __init__(self,window=None,canvas=None,name=None):
        # if window==None:return
        self.cam = cv.VideoCapture(0)
        self.canvas= canvas.getCanvas1()
        self.clear = canvas.clear_console
        self.name=name
        self.list_image=[]
        self.face = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
        self.stop=0
        self.captured_button = tk.Button(window,text="Stop",command=lambda: self.stop_fuc())
        self.captured_button.place(x=1350,y=540)
        self.numberOfImage = 0
        self.numberOfLoop = 0
    def stop_fuc(self):self.stop=1
    def capture(self): 
        check,frame = self.cam.read()
        # saved_frame=frame
        frame = cv.resize(frame,(1183, 733))
        if check:
            gray=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
            faces = self.face.detectMultiScale(gray,scaleFactor=1.2,minNeighbors=5)
            for (x,y,w,h) in faces:
                cut_face = gray[y:y+h, x:x+w]
                cut_face = cv.resize(cut_face,(128,128))
                if self.numberOfLoop %15==0:
                    self.numberOfImage+=1
                    self.list_image.append(cut_face)
                cv.putText(frame,self.name, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv.rectangle(frame,(x, y), (x+w, y+h), (0, 255, 0), 2)
            self.image = cv.cvtColor(frame, cv.COLOR_BGR2RGB) # to RGB
            self.image = Image.fromarray(self.image) # to PIL format
            self.image = ImageTk.PhotoImage(image=self.image) # to ImageTk format
            # Update image
            self.canvas.create_image(4,55,anchor = tk.NW, image= self.image)
            self.canvas.img_tk= self.image
            #check i to return
            if(self.stop or self.numberOfImage>=100):
                self.cam.release()
                self.clear()
                self.captured_button.destroy()
                self.Train(self.name,self.list_image)
                return
            # Repeat every 'interval' ms
            self.canvas.after(50,self.capture)
    def Train(self,name,list_image):
        all_name_path = [os.path.join("DataSet1",f) for f in os.listdir("DataSet1")]
        all_name = [x.split('\\')[-1] for x in all_name_path]
        if name not in all_name:
            os.mkdir("DataSet1\\" + name)
        max = 0
        for x in os.listdir("DataSet1\\"+name):
            x=x.split("(")[-1].split(")")[0]
            if int(x)>max:max = int(x)
        max +=1
        for image in list_image:
            cv.imwrite("DataSet1\\"+name + "\\"+name+"("+str(max)+")"+".jpg",image)
            max+=1
        train()
    def pick_picture(self):
        image_path = fd.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        image = cv.imread(image_path)
        self.save_image(image_path.split('/')[-1].split('.')[0],image)
        

In [105]:
def button2_clicked(window,entry,canvas):
    canvas.clear_console()
    name=entry.get()
    if name == "":return
    a = add_person(window,canvas,name)
    a.capture()


In [106]:
class show_image():
    def __init__(self,window,canvas2):
        self.canvas2 =canvas2
        self.window =window
        self.attendance=[]
        self.detect_face = cv.CascadeClassifier("haarcascade_frontalface_default.xml")
    def load_data(self):
        self.model = load_model("my_model.keras")
        self.all_name = [name for name in os.listdir("DataSet1")]
        print(self.all_name)
        path = fd.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")])
        self.image = cv.imread(path)
    def show_image(self):
        self.load_data()
        gray_image = cv.cvtColor(self.image,cv.COLOR_BGR2GRAY)
        faces = self.detect_face.detectMultiScale(gray_image,scaleFactor=1.2,minNeighbors=5)
        for (x,y,w,h) in faces:
                cv.rectangle(self.image,(x, y), (x+w, y+h), (0, 255, 0), 2)
                cut_face = gray_image[y:y+h, x:x+w]
                cut_face = cv.resize(cut_face,(128,128))
                predict = self.model.predict(np.expand_dims(cut_face,axis=0))
                index = np.argmax(predict)
                print(predict)
                name = self.all_name[index]
                cv.putText(self.image,name, (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        img = Image.fromarray(cv.cvtColor(self.image,cv.COLOR_BGR2RGB))
        img = img.resize((1183, 733), Image.LANCZOS)
        img=ImageTk.PhotoImage(img)
        panel = tk.Label(self.window, image=img)
        panel.image = img
        panel.place(x=4,y=55)
        cv.destroyAllWindows()
        

In [110]:
if __name__== '__main__':
    interface()
    

Epoch 1/10
15/15 [==============================] - 10s 614ms/step - loss: 0.9967 - accuracy: 0.4922
Epoch 2/10
15/15 [==============================] - 9s 574ms/step - loss: 0.8362 - accuracy: 0.6414
Epoch 3/10
15/15 [==============================] - 9s 578ms/step - loss: 0.4964 - accuracy: 0.8241
Epoch 4/10
15/15 [==============================] - 9s 570ms/step - loss: 0.2145 - accuracy: 0.9354
Epoch 5/10
15/15 [==============================] - 9s 572ms/step - loss: 0.2444 - accuracy: 0.9042
Epoch 6/10
15/15 [==============================] - 9s 574ms/step - loss: 0.1366 - accuracy: 0.9555
Epoch 7/10
15/15 [==============================] - 9s 574ms/step - loss: 0.1336 - accuracy: 0.9332
Epoch 8/10
15/15 [==============================] - 9s 564ms/step - loss: 0.1132 - accuracy: 0.9488
Epoch 9/10
15/15 [==============================] - 9s 570ms/step - loss: 0.0706 - accuracy: 0.9733
Epoch 10/10
15/15 [==============================] - 9s 575ms/step - loss: 0.0678 - accuracy: 0.964